In [ ]:
import pandas as pd
# Ajuste das configurações de exibição
pd.set_option('display.float_format', lambda x: '%.2f' % x)
########################################## LENDO A BASE OPERACIONAL PRONTA PARA TRABALHAR
caminho = r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\BASES SEMANA PADRAO\BASE_OPERACIONAL"
base_operacional_hugo = pd.read_parquet(caminho+"/BASE_OPERACIONAL_TELECOB_HUGO.parquet")

base_operacional_hugo['ATRASO'] = base_operacional_hugo['ATRASO'].astype('int16')
base_operacional_hugo['SALDO_ABERTO'] = base_operacional_hugo['SALDO_ABERTO'].str.replace('.', '').str.replace(',', '.').astype(float)
base_operacional_hugo['BASE'] = base_operacional_hugo['BASE'].apply(lambda x: x.replace('Claro MÃ³vel','Claro Móvel'))
base_operacional_hugo = base_operacional_hugo.assign(FILA="")

## Separando telefones contratante e higi
base_contratante = base_operacional_hugo.loc[:,['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE']]
base_higi = base_operacional_hugo.loc[:,['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE_HIGIENIZADO']]

#### tirando dos telefones higienizados os dados nulos
base_higi = base_higi.query("TELEFONE_HIGIENIZADO.notnull()")
base_higi = base_higi.rename(columns={'TELEFONE_HIGIENIZADO':'TELEFONE'})

## unificando a base em layout vertical
base_unificada = pd.concat([base_contratante,base_higi]).drop_duplicates(['CPF','NOME_x','DEVEDOR','BASE','TELEFONE'])
base_unificada['TELEFONE'] = base_unificada['TELEFONE'].str.strip()


#### LISTA DOS STATUS DE TELEFONIA PARA DIVISÃO DO MAILING
status_alo = ['vozhumana','ocupado','naoatende','cxpre','cxpos','timeout']
nao_validado = ['nao_validado']
status_telefonia = ['congestionamentopublica','foneinexistente','foneforadeservico','silencio','semrespostapublica','congestionamentointerno','congestionamentopf']

In [ ]:
### CARREGANDO AS DISCAGENS DA ATN GERAL
telefones_validados = pd.read_parquet(r"R:\TI\TELEFONIA\validados_2024/telefones_validados_full.parquet")
telefones_validados.rename(columns={'ResultadoClassificacao':'STATUS_TEL'},inplace=True)
telefones_validados['dddtelefone'] = telefones_validados['dddtelefone'].astype('str')

In [ ]:
##### CRUZANDO A BASE OPERACIONAL E TRAZENDO OS STATUS DE TELEFONIA 
base_unificada = pd.merge(base_unificada, telefones_validados, right_on=['dddtelefone'], left_on=['TELEFONE'], how="left")
base_unificada = base_unificada.drop(columns='dddtelefone')

### marcando oq nao foi validado
base_unificada['STATUS_TEL'] = base_unificada['STATUS_TEL'].fillna( 'nao_validado')

## MAILINGS PF

In [ ]:
### FILTRANDO A BASE DE PF
base_unificada_pf = base_unificada[base_unificada['CPF'].str.len()==11]
base_unificada_pf = base_unificada_pf.drop_duplicates(['CPF','DEVEDOR','BASE','TELEFONE'])

In [ ]:
alo_net = base_unificada_pf.query(f"STATUS_TEL in {status_alo} and BASE == 'GEVENUE NET'")[['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE']]

nao_validado_net = base_unificada_pf.query(f"STATUS_TEL in {nao_validado} and BASE == 'GEVENUE NET'")[['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE']]

status_telefonia_net = base_unificada_pf.query(f"STATUS_TEL in {status_telefonia} and BASE == 'GEVENUE NET'")[['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE']]


alo_net.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO E NET PF/alo_net.csv",index=False,sep=';')
nao_validado_net.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO E NET PF/nao_validado_net.csv",index=False,sep=';')
status_telefonia_net.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO E NET PF/status_telefonia_net.csv",index=False,sep=';')

In [ ]:
alo_movel = base_unificada_pf.query(f"STATUS_TEL in {status_alo} and BASE == 'GEVENUE CLARO MOVEL'")[['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE']]

nao_validado_movel = base_unificada_pf.query(f"STATUS_TEL in {nao_validado} and BASE == 'GEVENUE CLARO MOVEL'")[['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE']]

status_telefonia_movel = base_unificada_pf.query(f"STATUS_TEL in {status_telefonia} and BASE == 'GEVENUE CLARO MOVEL'")[['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE']]

alo_movel.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO E NET PF/alo_movel.csv",index=False,sep=';')
nao_validado_movel.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO E NET PF/nao_validado_movel.csv",index=False,sep=';')
status_telefonia_movel.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO E NET PF/status_telefonia_movel.csv",index=False,sep=';')

In [ ]:
alo_fixo = base_unificada_pf.query(f"STATUS_TEL in {status_alo} and BASE == 'GEVENUE CLARO FIXO'")[['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE']]
nao_validado_fixo = base_unificada_pf.query(f"STATUS_TEL in {nao_validado} and BASE == 'GEVENUE CLARO FIXO'")[['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE']]
status_telefonia_fixo = base_unificada_pf.query(f"STATUS_TEL in {status_telefonia} and BASE == 'GEVENUE CLARO FIXO'")[['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE']]

alo_fixo.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO E NET PF/alo_fixo.csv",index=False,sep=';')
nao_validado_fixo.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO E NET PF/nao_validado_fixo.csv",index=False,sep=';')
status_telefonia_fixo.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO E NET PF/status_telefonia_fixo.csv",index=False,sep=';')

In [ ]:
alo_tv = base_unificada_pf.query(f"STATUS_TEL in {status_alo} and BASE == 'GEVENUE CLARO TV'")[['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE']]
nao_validado_tv = base_unificada_pf.query(f"STATUS_TEL in {nao_validado} and BASE == 'GEVENUE CLARO TV'")[['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE']]
status_telefonia_tv = base_unificada_pf.query(f"STATUS_TEL in {status_telefonia} and BASE == 'GEVENUE CLARO TV'")[['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE']]

alo_tv.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO E NET PF/alo_tv.csv",index=False,sep=';')
nao_validado_tv.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO E NET PF/nao_validado_tv.csv",index=False,sep=';')
status_telefonia_tv.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO E NET PF/status_telefonia_tv.csv",index=False,sep=';')

## MAILINGS PJ

In [ ]:
### FILTRANDO A BASE DE PJ
base_unificada_pj = base_unificada[base_unificada['CPF'].str.len()==14]
base_unificada_pj = base_unificada_pj.drop_duplicates(['CPF','DEVEDOR','BASE','TELEFONE'])

In [ ]:
alo_net_pj = base_unificada_pj.query(f"STATUS_TEL in {status_alo} and BASE == 'GEVENUE NET'")[['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE']]

nao_validado_net_pj = base_unificada_pj.query(f"STATUS_TEL in {nao_validado} and BASE == 'GEVENUE NET'")[['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE']]

status_telefonia_net_pj = base_unificada_pj.query(f"STATUS_TEL in {status_telefonia} and BASE == 'GEVENUE NET'")[['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE']]


alo_net_pj.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO E NET PJ/alo_net_pj.csv",index=False,sep=';')
nao_validado_net_pj.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO E NET PJ/nao_validado_net_pj.csv",index=False,sep=';')
status_telefonia_net_pj.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO E NET PJ/status_telefonia_net_pj.csv",index=False,sep=';')

In [ ]:
alo_movel_pj = base_unificada_pj.query(f"STATUS_TEL in {status_alo} and BASE == 'GEVENUE CLARO MOVEL'")[['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE']]

nao_validado_movel_pj = base_unificada_pj.query(f"STATUS_TEL in {nao_validado} and BASE == 'GEVENUE CLARO MOVEL'")[['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE']]

status_telefonia_movel_pj = base_unificada_pj.query(f"STATUS_TEL in {status_telefonia} and BASE == 'GEVENUE CLARO MOVEL'")[['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE']]

alo_movel_pj.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO E NET PJ/alo_movel_pj.csv",index=False,sep=';')
nao_validado_movel_pj.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO E NET PJ/nao_validado_movel_pj.csv",index=False,sep=';')
status_telefonia_movel_pj.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO E NET PJ/status_telefonia_movel_pj.csv",index=False,sep=';')

In [ ]:
alo_fixo_pj = base_unificada_pj.query(f"STATUS_TEL in {status_alo} and BASE == 'GEVENUE CLARO FIXO'")[['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE']]
nao_validado_fixo_pj = base_unificada_pj.query(f"STATUS_TEL in {nao_validado} and BASE == 'GEVENUE CLARO FIXO'")[['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE']]
status_telefonia_fixo_pj = base_unificada_pj.query(f"STATUS_TEL in {status_telefonia} and BASE == 'GEVENUE CLARO FIXO'")[['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE']]

alo_fixo_pj.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO E NET PJ/alo_fixo_pj.csv",index=False,sep=';')
nao_validado_fixo_pj.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO E NET PJ/nao_validado_fixo_pj.csv",index=False,sep=';')
status_telefonia_fixo_pj.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO E NET PJ/status_telefonia_fixo_pj.csv",index=False,sep=';')

In [ ]:
alo_tv_pj = base_unificada_pj.query(f"STATUS_TEL in {status_alo} and BASE == 'GEVENUE CLARO TV'")[['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE']]
nao_validado_tv_pj = base_unificada_pj.query(f"STATUS_TEL in {nao_validado} and BASE == 'GEVENUE CLARO TV'")[['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE']]
status_telefonia_tv_pj = base_unificada_pj.query(f"STATUS_TEL in {status_telefonia} and BASE == 'GEVENUE CLARO TV'")[['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE']]

alo_tv_pj.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO E NET PJ/alo_tv_pj.csv",index=False,sep=';')
nao_validado_tv_pj.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO E NET PJ/nao_validado_tv_pj.csv",index=False,sep=';')
status_telefonia_tv_pj.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO E NET PJ/status_telefonia_tv_pj.csv",index=False,sep=';')

#### EMAILS TELECOB

In [ ]:
### lendo base de cobrança GEVENUE e carregando pelo INDEX somente CPF E EMAILS
import os
import pandas as pd
import glob

# Configuração para exibir todas as colunas no DataFrame
pd.set_option("display.max_columns", None)

# Definindo os nomes das colunas
header_novo_gevenue = [
    "CÓDIGO DA CONTA", "SENHA", "PRODUTO", "NOME", "TIPO DE PESSOA", "CPF/CNPJ",
    "DATA_NASCIMENTO", "FILA_COBRANCA", "ULT_DATA_SMS", "ULT_DATA_CARTA",
    "ULT_DATA_URA", "ULT_DATA_EMAIL", "CARTEIRA (REGIÃO)", "VENCIMENTO",
    "Valor Claro Móvel", "Valor Claro Residencial", "Valor Claro Empresarial",
    "Valor Claro Fixo", "Valor Claro TV", "Valor TvExpurgo", "Valor Oi",
    "DATA_NEGATIVACAO", "DATA_CADASTRO", "STATUS_CONTRATO_NET", "EMAIL_01",
    "EMAIL_02", "EMAIL_03", "TIPO_DE_ENDERECO_1", "ORIGEM_ENDERECO_1",
    "CIDADE_END_INS", "ESTADO_END_INS", "CEP_END_INS", "TIPO_DE_ENDERECO_2",
    "ORIGEM_ENDERECO_2", "CIDADE_END_FAT", "UF_END_FAT", "CEP_END_FAT",
    "DDD_FIXO_1", "TEL_FIXO_1", "DDD_FIXO_2", "TEL_FIXO_2", "DDD_FIXO_3",
    "TEL_FIXO_3", "DDD_FIXO_4", "TEL_FIXO_4", "DDD_CEL_1", "TEL_CEL_1",
    "DDD_CEL_2", "TEL_CEL_2", "DDD_CEL_3", "TEL_CEL_3", "DDD_CEL_4", "TEL_CEL_4"
]

# Definindo o caminho para a pasta onde estão os arquivos .txt
caminho = r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\BASE DE COBRANÇA\Base de Cobrança GEV"
arquivos = glob.glob(os.path.join(caminho, "*.txt"))

# Lista de larguras do layout NET COB
lista_de_larguras = [
    15, 12, 15, 40, 8, 15, 10, 50, 16, 16, 16, 16, 30, 10, 12, 12, 12, 12, 12, 12,
    12, 10, 10, 50, 50, 50, 50, 10, 300, 40, 2, 9, 10, 300, 40, 2, 9, 2, 10, 2,
    10, 2, 10, 2, 10, 2, 10, 2, 10, 2, 10, 2, 10, 2, 10, 2, 10
]

# Definindo os índices das colunas de interesse
colunas_de_interesse_indices = [5,  24, 25, 26]

# Selecionando as larguras e os nomes das colunas de interesse
larguras_interesse = [lista_de_larguras[i] for i in colunas_de_interesse_indices]
colunas_interesse = [header_novo_gevenue[i] for i in colunas_de_interesse_indices]

# Calculando as larguras cumulativas para pular colunas desnecessárias
sum_larguras = 0
colunas_interesse_larguras = []
for i in range(len(lista_de_larguras)):
    if i in colunas_de_interesse_indices:
        colunas_interesse_larguras.append((sum_larguras, lista_de_larguras[i]))
    sum_larguras += lista_de_larguras[i]

# Função personalizada para ler apenas as colunas de interesse
def ler_colunas_interesse(linha):
    valores = []
    for inicio, largura in colunas_interesse_larguras:
        valores.append(linha[inicio:inicio+largura].strip())
    return valores

# Lista para armazenar os DataFrames de cada arquivo
dfList = []

# Iterar sobre os arquivos na pasta
for arquivo in arquivos:
    print(f"Lendo arquivo: {arquivo}")
    
    # Lendo o arquivo e selecionando as colunas de interesse
    with open(arquivo, 'r', encoding='utf-8') as file:
        dados = [ler_colunas_interesse(linha) for linha in file]

    # Convertendo os dados para um DataFrame do pandas
    df = pd.DataFrame(dados, columns=colunas_interesse)
    dfList.append(df)

# Concatenar todos os DataFrames em um único DataFrame
df_completo = pd.concat(dfList, ignore_index=True)

email_cob_1 = df_completo[['CPF/CNPJ','EMAIL_01']]
email_cob_2 = df_completo[['CPF/CNPJ','EMAIL_02']]
email_cob_3 = df_completo[['CPF/CNPJ','EMAIL_03']]
email_cob_1 = email_cob_1.rename(columns={'CPF/CNPJ':'CPF','EMAIL_01':'EMAIL_BASE_COB'})
email_cob_2 = email_cob_2.rename(columns={'CPF/CNPJ':'CPF','EMAIL_02':'EMAIL_BASE_COB'})
email_cob_3 = email_cob_3.rename(columns={'CPF/CNPJ':'CPF','EMAIL_03':'EMAIL_BASE_COB'})

### concatenando os emails 1,2,3
emails_base_cobranca = pd.concat([email_cob_1,email_cob_2,email_cob_3])
## Removendo os emails em branco e tirando duplicatas
emails_base_cobranca = emails_base_cobranca.query("EMAIL_BASE_COB != ''").drop_duplicates(['CPF','EMAIL_BASE_COB'])

### limpando os dfs pra nao pesar na memoria
email_cob_1= ''
df_completo= ''
email_cob_2= ''
email_cob_3= ''

In [ ]:
########################################## LENDO A BASE OPERACIONAL PRONTA PARA TRABALHAR
caminho = r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\BASES SEMANA PADRAO\BASE_OPERACIONAL"
base_operacional_hugo = pd.read_parquet(caminho+"/BASE_OPERACIONAL_TELECOB_HUGO.parquet")

base_operacional_hugo['ATRASO'] = base_operacional_hugo['ATRASO'].astype('int16')
base_operacional_hugo['SALDO_ABERTO'] = base_operacional_hugo['SALDO_ABERTO'].str.replace('.', '').str.replace(',', '.').astype(float)
base_operacional_hugo['BASE'] = base_operacional_hugo['BASE'].apply(lambda x: x.replace('Claro MÃ³vel','Claro Móvel'))

###### BAIXANDO OS EMAILS DA GEVENUE HIGIENIZADOS (DAIANA PEDE PRA AÇÃO MASSIVA)
bases_gevenue_list = ['GEVENUE NET','GEVENUE CLARO MOVEL','GEVENUE CLARO FIXO','GEVENUE CLARO TV']

## SEPARANDO OS EMAILS HIGIENIZADOS 
emails_gevenue_higienizados = base_operacional_hugo.query(f"BASE in {bases_gevenue_list} and  EMAIL_HIGIENIZADO.notnull()").loc[:,['CPF','NOME_x','EMAIL_HIGIENIZADO','ATRASO','BASE','CONTRATO']]\
.drop_duplicates(['CPF','NOME_x','EMAIL_HIGIENIZADO'])

##### SEPARANDO OS EMAILS DO CONTRATANTE
emails_gevenue_contratante = base_operacional_hugo.query(f"BASE in {bases_gevenue_list} and  EMAIL.notnull()").loc[:,['CPF','NOME_x','EMAIL','ATRASO','BASE','CONTRATO']]\
.drop_duplicates(['CPF','NOME_x','EMAIL'])


###RENOMEANDO PARA CONCATENAR OS DOIS HIGIENIZADOS E CONTRATANTE
emails_gevenue_higienizados.rename(columns={'EMAIL_HIGIENIZADO': 'EMAIL'}, inplace=True)

# CONCATENANDO, REMOVENDO DUPLICADO E RESETANDO O INDEX
emails_gevenue_devolver = pd.concat([emails_gevenue_higienizados,emails_gevenue_contratante])
emails_gevenue_devolver = emails_gevenue_devolver.drop_duplicates(['CPF','NOME_x','EMAIL','CONTRATO']).reset_index(drop=True)

#########
## Trazendo os emails da base de cobrança para a base da semana
emails_gevenue_devolver = pd.merge(emails_gevenue_devolver, emails_base_cobranca, right_on=['CPF'], left_on=['CPF'], how="left")

emails_base_mailing = emails_gevenue_devolver[['CPF', 'NOME_x', 'EMAIL', 'ATRASO', 'BASE', 'CONTRATO']]
emails_base_cob = emails_gevenue_devolver[['CPF', 'NOME_x', 'EMAIL_BASE_COB', 'ATRASO', 'BASE', 'CONTRATO']]
emails_base_cob = emails_base_cob.rename(columns={'EMAIL_BASE_COB':'EMAIL'})

emails_gevenue_devolver = pd.concat([emails_base_mailing, emails_base_cob])
## Removendo duplicata e os dados nulos
emails_gevenue_devolver = emails_gevenue_devolver.drop_duplicates(['CPF','EMAIL','BASE','CONTRATO']).query("EMAIL.notnull()")


In [ ]:
###### FILTRANDO O QUE É = CPF (11 DIGITOS) PORQUE A QUANTIDDE ULTRAPASSA 1 MILHÃO DE LINHAS
### ORDENANDO POR ATRASO DO MENOR PARA O MAIOR E CPF , FILTRANDO SOMENTE AS LINHAS DE EMAIL QUE NÃO ESTÃO VAZIAS.
## DIVIDINDO EM 2 ARQUIVOS PARA ABRIREM EM EXCEL
emails_gevenue_devolver_cpf = emails_gevenue_devolver[emails_gevenue_devolver['CPF'].str.len()==11].reset_index(drop=True)
emails_gevenue_devolver_cpf = emails_gevenue_devolver_cpf.sort_values(['ATRASO','CPF']).query("EMAIL !=' '")
emails_gevenue_cpf_1 = emails_gevenue_devolver_cpf.iloc[0:999999]
emails_gevenue_cpf_2 = emails_gevenue_devolver_cpf.iloc[1000000:2000000]
emails_gevenue_cpf_3 = emails_gevenue_devolver_cpf.iloc[2000001:3000000]

###### FILTRANDO O QUE É = CNPJ (14 DIGITOS).
#### ORDENANDO POR ATRASO DO MENOR PARA O MAIOR E CPF , FILTRANDO SOMENTE AS LINHAS DE EMAIL QUE NÃO ESTÃO VAZIAS.
emails_gevenue_devolver_cnpj = emails_gevenue_devolver[emails_gevenue_devolver['CPF'].str.len()==14].reset_index(drop=True)
emails_gevenue_devolver_cnpj = emails_gevenue_devolver_cnpj.sort_values(['ATRASO','CPF']).query("EMAIL !=' '")



##### BAIXANDO O ARQUIVO COM 3 ABAS, DUAS CONTENDO EMAILS DE CLIENTES PESSOA FISICA E UMA COM PESSOA JURIDICA.
writer = pd.ExcelWriter(r'R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\EMAILS PF E PJ/Emails_PF_PJ_TELES.xlsx', engine='xlsxwriter')

#Salvando o Relatorio na pasta.
emails_gevenue_cpf_1.to_excel(writer, sheet_name='Emails_PF_1', index=False)
emails_gevenue_cpf_2.to_excel(writer, sheet_name='Emails_PF_2', index=False)
emails_gevenue_cpf_3.to_excel(writer, sheet_name='Emails_PF_3', index=False)
emails_gevenue_devolver_cnpj.to_excel(writer, sheet_name='Emails_PJ', index=False)
writer.save()
